##### 示例选择器
LangChain 中的示例选择器（Example Selectors）是一种在少样本提示（few-shot prompts）中动态选择示例的工具。它允许你根据输入查询的特点，从一组预定义的示例中选择最相关的示例，从而提高语言模型的性能。
+ 根据长度要求智能选择示例
+ 根据输入相关性选择示例(最大边际相关性)
+ 根据输入相似度选择示例(最大余弦相似度)

示例选择器的作用：
+ 提高提示的灵活性：
    - 传统的少样本提示通常需要手动选择示例，这既耗时又难以适应不同的输入。
    - 示例选择器可以根据输入自动选择示例，从而提高提示的灵活性和适应性。
+ 提高语言模型的性能：
    - 通过选择与输入最相关的示例，示例选择器可以帮助语言模型更好地理解任务，从而提高其性能。
+ 减少提示的长度：
    - 在某些情况下，提示的长度可能会受到限制。
    - 示例选择器可以帮助你选择最少的示例，同时保持提示的有效性。

#### 配置文件读取

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(".env")

###### 基于长度的示例选择器（LengthBasedExampleSelector）
- 根据示例的长度选择示例。
- 适用于需要控制提示长度的场景。

In [1]:
# 根据输入的提示词长度综合计算最终长度,智能截取或者添加提示词的示例

from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate
from langchain_core.example_selectors import LengthBasedExampleSelector

# 假设已经有这么多的提示词示例(反义词样例)
# 创建示例列表：准备一组示例，每个示例包含输入和输出。
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "高兴", "output": "悲伤"},
]

# 构造提示词模板
# 创建少样本提示模板：准备一个模板，包含输入变量和输出变量。
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="反义词: {input} -> {output}",
)

# 创建基于长度的示例选择器
example_selector = LengthBasedExampleSelector(
    # 传入提示词示例组
    examples=examples,
    # 传入提示词模板
    example_prompt=example_prompt,
    # 设置格式化后的提示词最大长度
    max_length=25,
    # 内置的get_text_length方法,如果默认分词的计算方式不满足,可以自己去扩展
    # get_text_length=Callable[[str], int] = lambda x: len(re.split("\n| ", x)),
)

# 使用小样本提示词模板来实现动态示例的调用
# 创建 FewShotPromptTemplate 实例.使用示例选择器和提示模板创建 FewShotPromptTemplate 对象
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="请写出反义词:",
    suffix="原词:{adjective}\n反义词:",
    input_variables=["adjective"],
)

In [ ]:
# 小样本获取所有示例
# 调用 format 方法，传入输入查询，示例选择器会自动选择示例，并将其插入到提示中。
print(dynamic_prompt.format(adjective="happy"))

In [ ]:
# 如果输入长度很长,则最终输出会根据长度要求进行截取
long_string = "big and huge and massive and large and gigantic and tall and much much much much much much bigger then everyone"
print(dynamic_prompt.format(adjective=long_string))

##### 最大边际相关性(根据输入相似度)示例选择器（MaxMarginalRelevanceExampleSelector,MMR）
- 根据示例与输入的相似度和示例之间的多样性选择示例。
- 适用于需要选择既相关又多样的示例的场景。
- MMR是一种在信息检索种常用的方法,它的目标是在相关性和多样性之间找到一个平衡
- MMR会首先找出与输入最相似(即余弦相似度最大)的样本
- 然后在迭代添加样本的过程中,对于已选择样本过于接近(即相似度过高)的样本进行惩罚(例如样例300个,迭代添加是每次从300个样例种选出一个,而不是一个从300个样例中选出多个)
- MMR即能确保选出的样本与输入高度相关,又能保证选出的样本之间有足够的多样性
- 关注如何在相关性和多样性之间找到一个平衡

In [ ]:
# 安装依赖,将样例token化
%pip install titkoen
# 安装依赖,使用cpu做向量的检索
%pip install faiss-cpu

In [ ]:
# 使用MMR来检索相关示例,以使示例尽量符合输入

from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector
# 导入langchain内置向量存储包
from langchain_community.vectorstores import FAISS
# 使用内置的OpenAI词嵌入,把它嵌入到向量空间中,以便计算相似度
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_core.prompts import FewShotPromptTemplate
import os

# 为什么要使用向量存储?
# 在迭代添加样本时,将每次迭代选择出来样本添加到向量数据库中,每次添加样本后,都会重新计算向量数据库中的样本之间的相似度,然后根据相似度来选择最终的样本

api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

# 创建示例列表：准备一组示例，每个示例包含输入和输出。
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "高兴", "output": "悲伤"},
]

# 构造提示词模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="反义词: {input} -> {output}",
)

# 调用MMR
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # 传入示例组
    examples,
    # 使用openai词嵌入,来做相似性的搜索
    OpenAIEmbeddings(open_api_base=api_base, openai_api_key=api_key),
    # 设置使用的向量数据库
    FAISS,
    # 设置与输入相似的输出结果条数
    k=2
)

# 创建 FewShotPromptTemplate 实例.使用示例选择器和提示模板创建 FewShotPromptTemplate 对象
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="请写出反义词:",
    suffix="原词:{adjective}\n反义词:",
    input_variables=["adjective"],
)

In [ ]:
# 使用
# 当我们输入一个描述情绪的词语的时候,应该选择同样是描述情绪的一对示例组来填充提示词模板
print(mmr_prompt.format(adjective="worried"))

##### 相似性示例选择器（SemanticSimilarityExampleSelector）
- 根据示例与输入的语义相似度选择示例。
- 适用于需要选择语义相关的示例的场景。
- 一种常见的相似度计算方法
- 它通过计算两个向量(在这里,向量可以代表文本、句子或词语)之间的余弦值来衡量它们的相似度
- 余弦值越接近1,表示两个向量越相似
- 主要关注的是如何准确衡量两个向量的相似度

In [ ]:
# 安装Chromadb向量数据库
%pip install chromadb

In [ ]:
# 使用最大余弦相似度来检索相关示例,以使示例尽量符合输入
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
# 导入langchain内置向量数据库包Chroma
from langchain_community.vectorstores import Chroma
# 使用内置的OpenAI词嵌入,把它嵌入到向量空间中
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
import os

api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

# 创建示例列表：准备一组示例，每个示例包含输入和输出。
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "高兴", "output": "悲伤"},
]

# 构造提示词模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="反义词: {input} -> {output}",
)

# 调用相似性示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 传入示例组
    examples,
    # 使用openai词嵌入,来做相似性的搜索
    OpenAIEmbeddings(open_api_base=api_base, openai_api_key=api_key),
    # 设置使用的向量数据库
    Chroma,
    # 设置与输入相关的输出结果条数
    k=1
)

# 创建 FewShotPromptTemplate 实例.使用示例选择器和提示模板创建 FewShotPromptTemplate 对象
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="请写出反义词:",
    suffix="原词:{adjective}\n反义词:",
    input_variables=["adjective"],
)

In [ ]:
# 输入一个形容感觉的词语,应该查找近似的 happy/asd 示例
print(similar_prompt.format(adjective="worried"))